In [14]:
# ===============================
# Step 0: Install & Import Libraries
# ===============================
# pip install transformers torch tensorflow scikit-learn pandas
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, GPT2Tokenizer, GPT2ForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [15]:

# ===============================
# Step 1: Generate Dataset
# ===============================
sentences = [
    "I love this movie", "This film was amazing", "I enjoyed every moment",
    "The acting was great", "What a fantastic experience",
    "Absolutely wonderful movie", "The story was touching", "I liked the characters",
    "Very entertaining film", "This movie made me happy",
    
    "I hate this movie", "This film was terrible", "I disliked every moment",
    "The acting was awful", "What a boring experience",
    "Absolutely horrible movie", "The story was weak", "I hated the characters",
    "Very disappointing film", "This movie made me angry"
] * 5  # 100 samples

labels = ["positive"] * 50 + ["negative"] * 50
df = pd.DataFrame({"sentence": sentences, "label": labels})

# Encode labels
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label"])  # positive=1, negative=0

# Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["sentence"], df["label_encoded"], test_size=0.2, random_state=42
)


In [16]:
# ===============================
# Step 2: LSTM Model
# ===============================
print("Training LSTM...")

# Tokenization & Padding
max_len = 6
lstm_tokenizer = Tokenizer()
lstm_tokenizer.fit_on_texts(df["sentence"])
X_train_seq = pad_sequences(lstm_tokenizer.texts_to_sequences(train_texts), maxlen=max_len, padding="post")
X_test_seq = pad_sequences(lstm_tokenizer.texts_to_sequences(test_texts), maxlen=max_len, padding="post")
y_train = np.array(train_labels)
y_test = np.array(test_labels)

# Build LSTM model
vocab_size = len(lstm_tokenizer.word_index) + 1
lstm_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=16, input_length=max_len),
    LSTM(32),
    Dense(1, activation="sigmoid")
])
lstm_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
lstm_model.fit(X_train_seq, y_train, epochs=5, batch_size=8, verbose=0)

# Evaluate LSTM
lstm_preds = (lstm_model.predict(X_test_seq) > 0.5).astype(int)
lstm_acc = accuracy_score(y_test, lstm_preds)
print(f"LSTM Accuracy: {lstm_acc:.4f}")

Training LSTM...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 871ms/step
LSTM Accuracy: 0.4000


In [17]:
# ===============================
# Step 3: BERT Model
# ===============================
print("Training BERT...")

# Tokenizer & Encodings
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = bert_tokenizer(list(train_texts), truncation=True, padding=True, max_length=32, return_tensors="pt")
test_encodings = bert_tokenizer(list(test_texts), truncation=True, padding=True, max_length=32, return_tensors="pt")

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, list(train_labels))
test_dataset = SentimentDataset(test_encodings, list(test_labels))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
bert_model.to(device)
optimizer = AdamW(bert_model.parameters(), lr=5e-5)

# Training loop
bert_model.train()
for epoch in range(3):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate BERT
bert_model.eval()
preds, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = bert_model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
bert_acc = accuracy_score(true_labels, preds)
print(f"BERT Accuracy: {bert_acc:.4f}")

Training BERT...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Accuracy: 0.6000


In [18]:
# ===============================
# Step 4: GPT Model
# ===============================
print("Training GPT-2...")

gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token
train_encodings = gpt_tokenizer(list(train_texts), truncation=True, padding=True, max_length=32, return_tensors="pt")
test_encodings = gpt_tokenizer(list(test_texts), truncation=True, padding=True, max_length=32, return_tensors="pt")

train_dataset = SentimentDataset(train_encodings, list(train_labels))
test_dataset = SentimentDataset(test_encodings, list(test_labels))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

gpt_model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
gpt_model.config.pad_token_id = gpt_model.config.eos_token_id
gpt_model.to(device)
optimizer = AdamW(gpt_model.parameters(), lr=5e-5)

# Training loop
gpt_model.train()
for epoch in range(3):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = gpt_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate GPT
gpt_model.eval()
preds, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = gpt_model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
gpt_acc = accuracy_score(true_labels, preds)
print(f"GPT-2 Accuracy: {gpt_acc:.4f}")

Training GPT-2...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT-2 Accuracy: 0.4500


In [19]:
# ===============================
# Step 5: Summary
# ===============================
print("\n✅ Accuracy Comparison:")
print(f"LSTM: {lstm_acc:.4f}")
print(f"BERT: {bert_acc:.4f}")
print(f"GPT-2: {gpt_acc:.4f}")



✅ Accuracy Comparison:
LSTM: 0.4000
BERT: 0.6000
GPT-2: 0.4500
